In [1]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F

Using backend: pytorch


In [2]:
import dgl.data

dataset = dgl.data.CoraGraphDataset()
print('Number of categories:', dataset.num_classes)

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Number of categories: 7


In [3]:
g = dataset[0]

In [4]:
print('Node features')
print(g.ndata)
print('Edge features')
print(g.edata)

Node features
{'train_mask': tensor([ True,  True,  True,  ..., False, False, False]), 'label': tensor([3, 4, 4,  ..., 3, 3, 3]), 'val_mask': tensor([False, False, False,  ..., False, False, False]), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True]), 'feat': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])}
Edge features
{}


In [5]:
from dgl.nn import GraphConv

class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

# Create the model with given dimensions
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)

In [6]:
g.ndata['feat'].shape[1]

1433

In [7]:
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(100):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)
train(g, model)

In epoch 0, loss: 1.946, val acc: 0.190 (best 0.190), test acc: 0.199 (best 0.199)
In epoch 5, loss: 1.891, val acc: 0.520 (best 0.630), test acc: 0.510 (best 0.634)
In epoch 10, loss: 1.809, val acc: 0.654 (best 0.654), test acc: 0.657 (best 0.657)
In epoch 15, loss: 1.704, val acc: 0.676 (best 0.676), test acc: 0.680 (best 0.680)
In epoch 20, loss: 1.577, val acc: 0.682 (best 0.684), test acc: 0.694 (best 0.692)
In epoch 25, loss: 1.429, val acc: 0.702 (best 0.702), test acc: 0.702 (best 0.701)
In epoch 30, loss: 1.264, val acc: 0.722 (best 0.722), test acc: 0.720 (best 0.720)
In epoch 35, loss: 1.092, val acc: 0.744 (best 0.744), test acc: 0.737 (best 0.731)
In epoch 40, loss: 0.920, val acc: 0.750 (best 0.750), test acc: 0.745 (best 0.745)
In epoch 45, loss: 0.759, val acc: 0.760 (best 0.760), test acc: 0.758 (best 0.756)
In epoch 50, loss: 0.616, val acc: 0.768 (best 0.768), test acc: 0.762 (best 0.762)
In epoch 55, loss: 0.495, val acc: 0.772 (best 0.772), test acc: 0.772 (best 0

In [8]:
g = g.to('cuda')
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes).to('cuda')
train(g, model)

In epoch 0, loss: 1.945, val acc: 0.146 (best 0.146), test acc: 0.149 (best 0.149)
In epoch 5, loss: 1.896, val acc: 0.412 (best 0.450), test acc: 0.441 (best 0.512)
In epoch 10, loss: 1.817, val acc: 0.626 (best 0.626), test acc: 0.638 (best 0.638)
In epoch 15, loss: 1.713, val acc: 0.686 (best 0.686), test acc: 0.727 (best 0.727)
In epoch 20, loss: 1.584, val acc: 0.682 (best 0.686), test acc: 0.719 (best 0.727)
In epoch 25, loss: 1.434, val acc: 0.684 (best 0.688), test acc: 0.716 (best 0.720)
In epoch 30, loss: 1.267, val acc: 0.700 (best 0.700), test acc: 0.731 (best 0.731)
In epoch 35, loss: 1.092, val acc: 0.716 (best 0.718), test acc: 0.752 (best 0.751)
In epoch 40, loss: 0.920, val acc: 0.722 (best 0.722), test acc: 0.749 (best 0.749)
In epoch 45, loss: 0.760, val acc: 0.730 (best 0.730), test acc: 0.752 (best 0.754)
In epoch 50, loss: 0.618, val acc: 0.730 (best 0.730), test acc: 0.755 (best 0.754)
In epoch 55, loss: 0.499, val acc: 0.732 (best 0.732), test acc: 0.756 (best 0